In [1]:
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
from pymatreader import read_mat
from scipy import sparse
import numpy as np
import os
from datetime import datetime, timedelta
import pandas
import cv2
import json
import pandas as pd
from amftrack.pipeline.paths.directory import *
from amftrack.util import *
import pickle
from joblib import Parallel, delayed
from datetime import datetime
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.post_processing.time_plate import *

In [2]:
directory = directory_project
update_analysis_info(directory)
analysis_info = get_analysis_info(directory)

In [3]:
select = analysis_info.loc[analysis_info['Plate']!=22]

In [4]:
frames = []
for index, row in select.iterrows():
    folder = row['folder_analysis']
    path_time_plate_info = row['path_time_plate_info']
    plate = row['Plate']
    if os.path.isfile(f'{directory}{path_time_plate_info}'):
        time_plate_info = pd.read_json(f'{directory}{path_time_plate_info}',
       convert_dates=True).transpose()
        time_plate_info.index.name = 't'
        time_plate_info.reset_index(inplace=True)
        for indexo, rowt in time_plate_info.iterrows():
            t = rowt['t']
            path_hyph_info_t = f'{directory}{folder}/time_hypha_info/hyph_info_{t}.json'
            if os.path.isfile(path_hyph_info_t):
                hyph_info_t = pd.read_json(path_hyph_info_t,
       convert_dates=True).transpose()
                hyph_info_t['Plate']=rowt['Plate']
                hyph_info_t['folder']=folder
                hyph_info_t['t']=rowt['t']
                frames.append(hyph_info_t)
time_hypha_infos = pd.concat(frames)

In [9]:
time_hypha_infos.loc[time_hypha_infos['density_window500'].isnull()]['Plate'].unique()

array([799, 792,  76, 808, 785])

In [5]:
time_hypha_infos.to_pickle('time_hypha_infos.pick')

In [14]:
time_hypha_infos.loc[(time_hypha_infos['Plate']==94)&(time_hypha_infos['end']==6992)]

,end,distance_final_pos,timedelta,time_since_emergence,speed,timestep,timestep_init,time_init,degree,width_tip_edge,av_width,has_reached_final_pos,in_ROI,width_root_edge,Plate,folder,t,density_window500,density_window1000,density_window2000
6992,6992,12016.9,1,1,212.64,23,23,23.15,1,8.17783,8.17783,False,True,NaN,94,Analysis_1636046622780809853_0_100_Version9,23,3271.88,NaN,NaN
6992,6992,11912.1,1,2,229.664,24,23,23.15,1,8.38372,8.38372,False,True,NaN,94,Analysis_1636046622780809853_0_100_Version9,24,3733.33,NaN,NaN
6992,6992,11792.7,1,3,230.319,25,23,23.15,1,8.54317,8.54317,False,True,NaN,94,Analysis_1636046622780809853_0_100_Version9,25,3928.12,NaN,NaN
6992,6992,11676.2,1,4,236.92,26,23,23.15,1,8.60524,8.60524,False,True,NaN,94,Analysis_1636046622780809853_0_100_Version9,26,4041.74,NaN,NaN
6992,6992,11550.3,1,5,244.542,27,23,23.15,1,8.60244,8.60244,False,True,NaN,94,Analysis_1636046622780809853_0_100_Version9,27,3744.93,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6992,6992,889.629,1,72.1,271.345,94,23,23.15,1,8.70408,10.2857,False,True,NaN,94,Analysis_1636046622780809853_0_100_Version9,94,1588.41,NaN,NaN
6992,6992,732.695,1,73.1,344.601,95,23,23.15,1,8.87801,10.285,False,True,NaN,94,Analysis_1636046622780809853_0_100_Version9,95,2096.23,NaN,NaN
6992,6992,535.8,1.01667,74.1167,282.659,96,23,23.15,1,8.5918,10.3319,False,True,NaN,94,Analysis_1636046622780809853_0_100_Version9,96,2316.52,NaN,NaN
6992,6992,369.611,0.983333,75.1,303.215,97,23,23.15,1,8.75792,10.3947,False,True,NaN,94,Analysis_1636046622780809853_0_100_Version9,97,1602.32,NaN,NaN


In [11]:
select = time_hypha_infos.loc[time_hypha_infos['Plate']==792]
hyphae = select['end'].unique()
select_hyphae = [hyph for hyph in hyphae if len(select.loc[select['end']==hyph])

In [12]:
hyphae

array([0, 1, 4, ..., 197833, 197843, 197853], dtype=object)